In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
import pickle
dunnhumby_path = "/Users/johnabraham/Desktop/dunnhumby_The-Complete-Journey/dunnhumby - The Complete Journey CSV"

In [2]:
trans_data = pd.read_csv(dunnhumby_path+"/transaction_data.csv")

In [3]:
# Make everything in lower case
trans_data.rename(columns=dict(zip(trans_data.columns,trans_data.columns.str.lower())),inplace=True)

In [4]:
trans_data_grpby = trans_data.groupby(['household_key','product_id'],as_index=False)['sales_value'].agg({'sales_value':'sum'})
#trans_data_cross = trans_data_grpby.pivot_table(index='household_key',columns='product_id', values='sales_value').reset_index()

In [30]:
hh_key_idx = dict(zip(trans_data_grpby.household_key.unique(),np.arange(len(trans_data_grpby.household_key.unique()))))
prod_id_idx = dict(zip(trans_data_grpby.product_id.unique(), np.arange(len(trans_data_grpby.product_id.unique()))))
prod_idx_2_id = dict(enumerate(trans_data_grpby.product_id.unique()))

In [6]:
hh_prod_mat = csr_matrix((trans_data_grpby.sales_value,
                          (list(map(lambda x: hh_key_idx[x],trans_data_grpby.household_key)),
                           list(map(lambda x: prod_id_idx[x],trans_data_grpby.product_id)))),
                          shape=(len(hh_key_idx),len(prod_id_idx)))

In [7]:
hh_prod_sim = cosine_similarity(hh_prod_mat.transpose())

In [8]:
# Earlier planned to write to a file to read in R. But the file is just too big (beyond 1 GB)
# hh_prod_sim_df = pd.DataFrame(hh_prod_sim)
# hh_prod_sim_df.columns=trans_data_grpby.product_id.unique()
# hh_prod_sim_df['product_id']=trans_data_grpby.product_id.unique()

In [9]:
# Output the file as a pickle file - Kernal dying
# with open("/Users/johnabraham/Desktop/dunnhumby_The-Complete-Journey/ProductRecommendation.P","wb") as file:
#     pickle.dump(hh_prod_sim_df,file)

In [53]:
### This part below expects the R-code to generate recommendation_output.csv is run and that file is available at this location
processed_files_path = "/Users/johnabraham/Dropbox/MSARelated/Spring2018/MGT6203/ProjectRelated"

'/Users/johnabraham/Dropbox/MSARelated/Spring2018/MGT6203/ProjectRelated/recommendation_output.csv'

In [11]:

# Read a dataset that has store_id, household_id, product_id from the historical data and use the collaborative filtering based method to append more products
# Return the dataset along with a key that says which one it is based on - historical or recommendation
reco_data = pd.read_csv(processed_files_path+"/recommendation_output.csv")
reco_data.head()

,store_id,household_key,recommendation_rank,product_id
0,292,559,5,838794
1,292,559,3,906345
2,292,559,2,908064
3,292,559,4,1106523
4,292,559,1,9487339


In [52]:
reco_data_augmented = reco_data[['store_id','household_key','product_id']]
num_reco_products = 5 # Only up to num_reco_products will be added
similarity_threshold = 0.5 # Only products with a similarity of similarity_threshold or above will be considered 'similar'
for _,record in reco_data.iterrows():
    prod_idx=prod_id_idx[record['product_id']]
    similarity_scores = hh_prod_sim[prod_idx,:]
    recommended_additional_product_idx = np.argsort(similarity_scores)[-(num_reco_products+1):-1]
    for prod_idx in recommended_additional_product_idx:
        if(similarity_scores[prod_idx]>=similarity_threshold):
            reco_data_augmented=reco_data_augmented.append(pd.DataFrame({"store_id":record['store_id'],"household_key":record['household_key'],
                                                                       'product_id':prod_idx_2_id[prod_idx]},index=[0]),ignore_index=True)
reco_data_augmented=reco_data_augmented.drop_duplicates()
reco_data_augmented.to_csv(processed_files_path+"/recommendation_augmented_output.csv",index=False)